In [2]:
import pandas as pd
import random

import geopandas as gpd 
from shapely.geometry import Point, Polygon
import h3

import plotly.express as px

import json

In [3]:
# Cargar datos geoespaciales en un DataFrame GeoPandas
gdf = gpd.read_file('data/df_italia.geojson')
gdf['geoid'] = gdf.index.astype(str)
gdf.geometry = gdf.geometry.to_crs(epsg = 4326) # 4326    # 3857
gdf.head()

,COD_UTS,DEN_UTS,h3,count,geometry,geoid
0,215,Milano,871f98241ffffff,22,"POLYGON ((8.86056 45.35089, 8.86204 45.33849, ...",0
1,215,Milano,871f98245ffffff,17,"POLYGON ((8.84271 45.36792, 8.84419 45.35552, ...",1
2,215,Milano,871f98248ffffff,37,"POLYGON ((8.90969 45.33698, 8.91115 45.32458, ...",2
3,215,Milano,871f98249ffffff,42,"POLYGON ((8.94097 45.34010, 8.94244 45.32769, ...",3
4,215,Milano,871f9824affffff,2,"POLYGON ((8.89625 45.31681, 8.89772 45.30440, ...",4


In [5]:
gdf['geoid'].min(), gdf['geoid'].max()

('0', '99')

In [6]:
gdf.shape

(367, 6)

In [24]:
df_punticos = pd.read_csv('data/df_punticos.csv', sep = ',', decimal = '.', header = 0, encoding = 'utf-8')
df_punticos = df_punticos[df_punticos['DEN_UTS'] == 'Milano']
df_punticos = df_punticos.iloc[0:100]
df_punticos['geo'] = df_punticos.index.astype(int)
df_punticos['geo2'] = df_punticos.index.astype(str)
df_punticos.head()

,latitude,longitude,DEN_UTS,COD_UTS,geo,geo2
524992,45.365934,9.348322,Milano,215,524992,524992
524993,45.472296,9.370329,Milano,215,524993,524993
524994,45.367242,9.314942,Milano,215,524994,524994
524995,45.446377,9.261899,Milano,215,524995,524995
524996,45.603352,8.939918,Milano,215,524996,524996


In [25]:
df_punticos.shape

(100, 6)

In [26]:
fig = px.choropleth_mapbox(gdf,
                           geojson=gdf.__geo_interface__,#['geometry']
                           locations=gdf.geoid,
                           color='count',
                           hover_data=["DEN_UTS"],
                           labels={'count':'Unidades'},
                           featureidkey='properties.geoid',
                           center={'lat': 45.48, 'lon':9.14},
                           mapbox_style='carto-positron',
                           zoom=10,
                           opacity=0.5,
                           )

# darle formato al toltip de los poligonos
# fig.update_traces(hovertemplate='<b>%{location}</b><br>Valor: %{z}')


# AGREGAR puntos

# Agregar la capa nueva como marcas de puntos
fig.add_scattermapbox(
    lat=df_punticos['latitude'],
    lon=df_punticos['longitude'],
    mode='markers',
    marker=dict(
        size=8,
        color=df_punticos['geo']
    ),
    text=df_punticos['DEN_UTS'] + '<br>' + df_punticos['geo2']
)










# Agregar evento de clic
fig.update_layout(
    clickmode='event+select',
    showlegend=False
)

def handle_click(trace, points, selector):
    if points.point_inds:
        selected_polygon = points.point_inds[0]
        print(f"Polígono seleccionado: {selected_polygon}")

fig.data[0].on_click(handle_click)









# Agrandar y mover labels

fig.update_layout(margin={'l': 0, 't': 0, 'r': 0, 'b': 0})
fig.update_coloraxes(
    colorbar=dict(
        lenmode="fraction",
        len=0.4,
        x=0,
        y=0.25
    )
)


fig.show()

In [6]:
fig = px.choropleth_mapbox(gdf,
                           geojson=gdf.__geo_interface__,#['geometry']
                           locations=gdf.geoid,
                           color='count',
                           hover_data=["DEN_UTS"],
                           labels={'count':'Unidades'},
                           featureidkey='properties.geoid',
                           center={'lat': 45.48, 'lon':9.14},
                           mapbox_style='carto-positron',
                           zoom=6)

# darle formato al toltip de los poligonos
# fig.update_traces(hovertemplate='<b>%{location}</b><br>Valor: %{z}')










# Agregar evento de clic
fig.update_layout(
    clickmode='event+select',
    showlegend=False
)

selected_polygons = []

def handle_click(trace, points, selector):
    if points.point_inds:
        selected_polygon = points.point_inds[0]
        if selected_polygon not in selected_polygons:
            selected_polygons.append(selected_polygon)
        else:
            selected_polygons.remove(selected_polygon)
        print(f"Polígonos seleccionados: {selected_polygons}")

fig.data[0].on_click(handle_click)





# Agrandar y mover labels

fig.update_layout(margin={'l': 0, 't': 0, 'r': 0, 'b': 0})
fig.update_coloraxes(
    colorbar=dict(
        lenmode="fraction",
        len=0.4,
        x=0,
        y=0.25
    )
)


fig.show()

In [ ]:
import cbsodata

In [ ]:
geodata_url = 'https://cartomap.github.io/nl/wgs84/gemeente_2023.geojson'
municipal_boundaries = gpd.read_file(geodata_url)
municipal_boundaries.head()

In [ ]:
data = pd.DataFrame(cbsodata.get_data('83765NED', select = ['WijkenEnBuurten', 'Codering_3', 'GeboorteRelatief_25']))
data['Codering_3'] = data['Codering_3'].str.strip()
data.head()

In [ ]:
# Link data from Statistics Netherlands to geodata
municipal_boundaries = pd.merge(municipal_boundaries, data,
                               left_on = "statcode", 
                               right_on = "Codering_3")
municipal_boundaries.head()

In [ ]:
#CRS: EPSG 3857 (web mercator projection wgs84)
municipal_boundaries.crs
# municipal_boundaries = municipal_boundaries.to_crs(epsg=3857)
municipal_boundaries = municipal_boundaries.to_crs(epsg=4326)

In [ ]:
#First column: Geoid, geometry column and  data columns
gdf_choro = municipal_boundaries.copy()
gdf_choro['geoid'] = gdf_choro.index.astype(str)
gdf_choro = gdf_choro[['geoid', 'geometry', 'statnaam', 'GeboorteRelatief_25']]
gdf_choro.head(3)

In [ ]:
type(gdf_choro)

In [ ]:
fig = px.choropleth_mapbox(gdf_choro,
                           geojson=gdf_choro.__geo_interface__,#['geometry']
                           locations=gdf_choro.geoid,
                           color='GeboorteRelatief_25',
                           featureidkey='properties.geoid',
                           center={'lat': 52.213, 'lon':5.2794},
                           mapbox_style='carto-positron',
                           zoom=6)
fig.show()

In [ ]:
fig = px.choropleth_mapbox(gdf_choro,
                           geojson=gdf_choro.__geo_interface__,#['geometry']
                           locations=gdf_choro.geoid,
                           color='GeboorteRelatief_25',
                           featureidkey='properties.geoid',
                           center={'lat': 52.213, 'lon':5.2794},
                           mapbox_style='carto-positron',
                           zoom=6)

# Agregar evento de clic
fig.update_layout(
    clickmode='event+select',
    showlegend=False
)

def handle_click(trace, points, selector):
    if points.point_inds:
        selected_polygon = points.point_inds[0]
        print(f"Polígono seleccionado: {selected_polygon}")

fig.data[0].on_click(handle_click)

fig.show()

In [ ]:
with open('data/df_italia.geojson', 'r') as infile:
    geo_json = json.load(infile)

In [ ]:
import pandas as pd
import plotly.express as px

# Datos de ejemplo del mapa de fondo
map_data = px.data.election()

# Datos de ejemplo de la capa nueva
point_data = pd.DataFrame({
    'Nombre': ['Punto D', 'Punto E', 'Punto F'],
    'Ciudad': ['Ciudad X', 'Ciudad Y', 'Ciudad Z'],
    'Latitud': [40.7128, 34.0522, 51.5074],
    'Longitud': [-74.0060, -118.2437, -0.1278],
    'Caracteristica': ['D', 'E', 'F']
})

# Crear el gráfico del mapa de fondo
fig = px.choropleth_mapbox(map_data, 
                           geojson=map_data.__geo_interface__,
                           locations=map_data.index,
                           color='winner',
                           mapbox_style="carto-positron",
                           zoom=3, 
                           center={"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'winner':'Winner'}
                          )

# Agregar la capa nueva como marcas de puntos
fig.add_scattermapbox(
    lat=point_data['Latitud'],
    lon=point_data['Longitud'],
    mode='markers',
    marker=dict(
        size=8,
        color='red'
    ),
    text=point_data['Nombre'] + '<br>' + point_data['Ciudad']
)

fig.update_layout(margin={'l': 0, 'r': 0, 't': 0, 'b': 0})  # Configuración de los márgenes

# Mostrar el gráfico
fig.show()
